In [25]:
from gurobipy import *

In [31]:
# Import necessary libraries
import json
import os

# Path to the input file generated by the Flask app
input_file = "inputs.json"

# Step 1: Load the input data
if os.path.exists(input_file):
    with open(input_file, "r") as f:
        data = json.load(f)
else:
    raise FileNotFoundError(f"{input_file} not found. Run the Flask app to save the inputs.")

# Extract values
warehousesNbre = data["warehousesNbre"]
customersNbre = data["customersNbre"]
fixed_costs = data["fixed_costs"]
transport_costs = data["transport_costs"]
variable_costs = data["variable_costs"]
penalty_costs = data.get("penalty_costs", [0] * customersNbre)  # Optional penalty costs
demands = data["demands"]
capacities = data["capacities"]
min_service = data.get("min_service", [1.0] * customersNbre)  # Default to 100% service level


warehouses = range(warehousesNbre)  # Index of warehouses
customers = range(customersNbre)  # Index of customers

# Fixed costs for opening each warehouse
#fixed_costs = list(map(int, input("Enter space-separated integers presenting fixed cost for each warehouse w: ").split()))
# Transportation costs: cost to serve each customer from each warehouse
#transport_costs = []
# for i in warehouses:
#     row = list(map(int, input(f"Enter space-separated values for row {i+1}: ").split()))
#     transport_costs.append(row)
# Step 4: Display the inputs
print("Number of Warehouses:", warehousesNbre)
print("Number of Customers:", customersNbre)
print("Fixed Costs:", fixed_costs)
print("Transportation Costs:")
for row in transport_costs:
    print(row)
print("Variable Costs:", variable_costs)
print("Penalty Costs:", penalty_costs)
print("Demands:", demands)
print("Capacities:", capacities)
print("Minimum Service Levels:", min_service)

# Customer demands (optional for extension, not used here directly)
#demands = [1, 1, 1, 1]  # Assume unit demand per customer

Number of Warehouses: 3
Number of Customers: 5
Fixed Costs: [1000, 1200, 900]
Transportation Costs:
[5, 10, 7, 6, 8]
[8, 5, 6, 9, 7]
[7, 8, 5, 7, 6]
Variable Costs: [1, 2, 1.5]
Penalty Costs: [10, 15, 12, 11, 14]
Demands: [300, 500, 400, 350, 450]
Capacities: [1000, 1200, 1100]
Minimum Service Levels: [0.8, 0.9, 0.85, 0.75, 0.9]


In [32]:
# Model
model = Model("Warehouse_Location")

# Decision variables
# Binary variables: 1 if warehouse w is open, 0 otherwise
open_warehouse = model.addVars(warehouses, vtype=GRB.BINARY, name="Open")

# Binary variables: 1 if customer c is served by warehouse w, 0 otherwise
serve_customer = model.addVars(warehouses, customers, vtype=GRB.INTEGER, name="Serve")


In [33]:
# Objective: Minimize total fixed and transportation costs
model.setObjective(
    quicksum(fixed_costs[w] * open_warehouse[w] for w in warehouses) +
    quicksum(transport_costs[w][c] * serve_customer[w, c] for w in warehouses for c in customers)+
    quicksum(variable_costs[r] * quicksum(serve_customer[r, d] for d in customers) for r in warehouses) +
    quicksum(penalty_costs[d] * (demands[d] - quicksum(serve_customer[r, d] for r in warehouses)) for d in customers),
    GRB.MINIMIZE
)
# Constraints

# 1. Each customer's demand is fully or partially satisfied
for c in customers:
    model.addConstr(quicksum(serve_customer[w, c] for w in warehouses) <= demands[c], f"Demand_{c}")

# 2. A warehouse can only serve a customer if it is open
for w in warehouses:
    for c in customers:
        model.addConstr(serve_customer[w, c] <= capacities[w] * open_warehouse[w], f"ServiceLink_{w}_{c}")

# 3. Total capacity of a warehouse cannot be exceeded
for w in warehouses:
    model.addConstr(quicksum(serve_customer[w, c] for c in customers) <= capacities[w], f"Capacity_{w}")

# 4. Minimum service level for each customer
for c in customers:
    model.addConstr(quicksum(serve_customer[w, c] for w in warehouses) >= min_service[c] * demands[c], f"MinService_{c}")

# 5. Non-negativity of served demand
model.addConstrs(serve_customer[w, c] >= 0 for w in warehouses for c in customers)

# 6. Unserved demand incurs penalty
for c in customers:
    model.addConstr(quicksum(serve_customer[w, c] for w in warehouses) <= demands[c], f"UnservedPenalty_{c}")

# Optimize the model
model.optimize()


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 48 rows, 18 columns and 105 nonzeros
Model fingerprint: 0xd8c3461d
Variable types: 0 continuous, 18 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 1e+03]
Found heuristic solution: objective 20199.000000
Presolve removed 20 rows and 0 columns
Presolve time: 0.00s
Presolved: 28 rows, 18 columns, 75 nonzeros
Variable types: 0 continuous, 18 integer (3 binary)

Root relaxation: objective 1.682500e+04, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0

In [34]:
# Print results
if model.status == GRB.OPTIMAL:
    print("\nOptimal Solution Found:\n")
    for w in warehouses:
        if open_warehouse[w].x > 0.5:  # Open warehouses
            print(f"Warehouse {w+1} is open.")
    for w in warehouses:
        for c in customers:
            if serve_customer[w, c].x > 0:
                print(f"Customer {c+1} is served by Warehouse {w+1} with {serve_customer[w, c].x} units.")
    print(f"\nTotal Cost = {model.objVal}")
else:
    print("No optimal solution found.")


Optimal Solution Found:

Warehouse 1 is open.
Warehouse 2 is open.
Warehouse 3 is open.
Customer 1 is served by Warehouse 1 with 300.0 units.
Customer 4 is served by Warehouse 1 with 350.0 units.
Customer 2 is served by Warehouse 2 with 500.0 units.
Customer 3 is served by Warehouse 3 with 400.0 units.
Customer 5 is served by Warehouse 3 with 450.0 units.

Total Cost = 16825.0
